In [71]:
import os
import sys
import numpy as np
from collections import defaultdict
import gmpy2
import copy
import logging

inputfilepath = 'C:\EMOLGUI\A_O_C\Day21.txt'
#inputfilepath = 'C:\EMOLGUI\A_O_C\Day21.sample.txt'

In [72]:
class Monkey():

    def __init__(self,name,operation,values):
        self.name = name
        self.operation = operation
        self.operationValue = values
        self.dependents = []
        if operation == 'Number':
            self.result = values
        else:
            self.result = None

    def operate(self, values):
        if self.operation == 'Number':
            return self.result
        else:
            if (values[0] == None) or (values[1] == None):
                return None
            if self.operation == '+':
                self.result = values[0] + values[1]
            if self.operation == '-':
                self.result = values[0] - values[1]
            if self.operation == '*':
                self.result = values[0] * values[1]
            if self.operation == '/':
                self.result = values[0] / values[1]
            if self.operation == '=':
                self.result = values[0] == values[1]
            return self.result

    def __repr__(self):
        return f'Monkey {self.name} with result {self.result}. It depends on {self.operationValue} and has dependents {self.dependents}'

    def setDependent(self, monkey):
        self.dependents.append(monkey)

    def refreshValue(self, monkeySet):
        if self.operation == 'Number':
            return self.result
        value1 = monkeySet[self.operationValue[0]].result
        value2 = monkeySet[self.operationValue[1]].result
        return self.operate([value1, value2])
    
    def cascadeUpdate(self, monkeySet):
        self.refreshValue(monkeySet)
        for upMonkey in self.dependents:
            monkeySet[upMonkey].cascadeUpdate(monkeySet)
    
    def updateValue(self, value, monkeySet):
        self.result = value
        self.cascadeUpdate(monkeySet)


In [73]:
monkeySet = {}
with open(inputfilepath,'r') as f:
    for line in f:
        linefields = line.rstrip().split(':')
        monkeyName = linefields[0]
        operation = linefields[1].lstrip().split()
        if len(operation) == 1:
            operationType = 'Number'
            operationValue = int(operation[0])
        else:
            operationType = operation[1]
            operationValue = [operation[0], operation[2]]
        monkey = Monkey(monkeyName,operationType,operationValue)
        monkeySet[monkeyName] = monkey

for monkeyName, monkeyObj in monkeySet.items():
    if monkeyObj.operation != 'Number':
        for downMonkey in monkeyObj.operationValue:
            monkeySet[downMonkey].setDependent(monkeyName)

In [74]:
monkeySet['humn']

Monkey humn with result 3951. It depends on 3951 and has dependents ['cpbh']

In [75]:
def getMonkeyValue(monkeySet, monkeyName):
    if monkeySet[monkeyName].result is not None:
        return monkeySet[monkeyName].result
    else:
        operand1 = monkeySet[monkeyName].operationValue[0]
        operand2 = monkeySet[monkeyName].operationValue[1]
        value1 = getMonkeyValue(monkeySet, operand1)
        value2 = getMonkeyValue(monkeySet, operand2)
        return monkeySet[monkeyName].operate([value1, value2])        

In [76]:
getMonkeyValue(monkeySet, 'root')

223971851179174.0

#### QUESTION 2

In [77]:
monkeySetQ2 = copy.deepcopy(monkeySet)
monkeySetQ2['root']

Monkey root with result 223971851179174.0. It depends on ['lntp', 'bjft'] and has dependents []

In [78]:
human = monkeySetQ2['humn']
monkeySetQ2['root'].operation = '-'
human.updateValue(5,monkeySetQ2)
getMonkeyValue(monkeySetQ2, 'humn'), getMonkeyValue(monkeySetQ2, 'root')

(5, 129529183963263.31)

In [130]:
# Manual testing numbers to reach 0 in 'root' based on MINUS operator (which is the same as True with an Equal)

testValue = 3_379_022_190_351
human.updateValue(testValue,monkeySetQ2)
getMonkeyValue(monkeySetQ2, 'humn'), getMonkeyValue(monkeySetQ2, 'root')

(3379022190351, 0.0)

In [70]:
# Brute Force is too long I'm afraid
humnTestValue = -1
human = monkeySetQ2['humn']
while not getMonkeyValue(monkeySetQ2, 'root'):
    humnTestValue += 1
    human.updateValue(humnTestValue,monkeySetQ2)


KeyboardInterrupt: 

In [61]:
humnTestValue

301